In [1]:
import mailbox
import csv
import re
from email.header import decode_header
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score



In [ ]:
# Ghép nhiều file lại với nhau

mbox_files = ['data1.mbox', 'data2.mbox', 'data3.mbox'] # tải file takeout của gg về giải nén ra rồi đổi tên hoặc 
output_file = 'data.mbox'
merged_mbox = mailbox.mbox(output_file)
for mbox_file in mbox_files:
    current_mbox = mailbox.mbox(mbox_file)
    

    for message in current_mbox:
        merged_mbox.add(message)
merged_mbox.flush()
merged_mbox.close()

print(f"File '{output_file}' đã được tạo thành công với tất cả các thư từ các file mbox đã cho.")


File 'data.mbox' đã được tạo thành công với tất cả các thư từ các file mbox đã cho.


In [3]:
def load_words(file_path):
    with open(file_path, encoding='utf-8') as f:
        return set(line.strip().lower() for line in f if line.strip())

stopwords = load_words('vietnamese-stopwords.txt')
spamwords = load_words('spamword.txt')

def decode_mime_words(s):
    if not s:
        return ''
    decoded_parts = decode_header(s)
    result = ''
    for part, enc in decoded_parts:
        if isinstance(part, bytes):
            try:
                result += part.decode(enc or 'utf-8', errors='ignore')
            except LookupError:
                result += part.decode('utf-8', errors='ignore')
        else:
            result += part
    return result


def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'[\n\r\t]+', ' ', text)
    text = re.sub(r'[^\w\sÀ-ỹ]', '', text)
    words = text.lower().split()
    filtered = [w for w in words if w not in stopwords]
    return ' '.join(filtered)

def extract_emails(mbox_file):
    data = []
    for message in mailbox.mbox(mbox_file):
        subject = decode_mime_words(message['subject'])
        subject_clean = clean_text(subject)

        body = ''
        if message.is_multipart():
            for part in message.walk():
                if part.get_content_type() == 'text/plain':
                    try:
                        body = part.get_payload(decode=True).decode(part.get_content_charset() or 'utf-8', errors='ignore')
                        break
                    except:
                        pass
        else:
            try:
                body = message.get_payload(decode=True).decode(message.get_content_charset() or 'utf-8', errors='ignore')
            except:
                pass

        body_clean = clean_text(body)
        full_text = subject_clean + ' ' + body_clean

        is_spam = 1 if any(word in full_text for word in spamwords) else 0 # make a spam label
        data.append((is_spam, full_text)) 
        output = "output.csv"
        with open(output, 'w', encoding='utf-8', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['label', 'email_content']) 
            writer.writerows(data) 
    return data

# Logistic Regression 
def train_model(data):
    labels,texts  = zip(*data)

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(texts)
    y = labels

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print("\n===== ĐÁNH GIÁ MÔ HÌNH =====")
    print(f"Tổng số email xử lý: {len(data)}")
    print(f"Số mẫu training     : {len(y_train)}")
    print(f"Số mẫu test         : {len(y_test)}")
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}\n")
    print(classification_report(y_test, y_pred, target_names=["Not Spam", "Spam"]))






In [4]:
mbox_path = "data2.mbox" 
emails = extract_emails(mbox_path)
train_model(emails)


===== ĐÁNH GIÁ MÔ HÌNH =====
Tổng số email xử lý: 1638
Số mẫu training     : 1310
Số mẫu test         : 328

Accuracy: 0.9268

              precision    recall  f1-score   support

    Not Spam       0.93      1.00      0.96       304
        Spam       0.00      0.00      0.00        24

    accuracy                           0.93       328
   macro avg       0.46      0.50      0.48       328
weighted avg       0.86      0.93      0.89       328



C:\Users\doanx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\doanx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\doanx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati